# TP4 Alejandro balladares 101118

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#1PbJBh7uDOaWqZ5GOrEFS3EnGNS4OgB58

id_silkroad_items = '1PbJBh7uDOaWqZ5GOrEFS3EnGNS4OgB58'
downloaded = drive.CreateFile({'id': id_silkroad_items})
downloaded.GetContentFile('silkroad_items.csv')

df_silkroad_items = pd.read_csv('silkroad_items.csv')
#df_silkroad_items.head().T

In [5]:
!pip install category_encoders

In [6]:

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model

from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
pd.options.display.max_columns = None


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Tarea 3: Regresión sobre el precio del ítem

¿Se puede predecir el precio del ítem en base a su información? ¿Qué tan bien funciona en
validación? ¿Qué tan bien funciona años después en versus?


In [7]:
def agregarFechas(df, columna):
  df[columna] = pd.to_datetime(df[columna])
  df['Fecha'] = df_silkroad_items[columna].transform(lambda x: x.date())

  df['Año'] = df['Fecha'].transform(lambda x: x.year)
  df['Mes'] = df['Fecha'].transform(lambda x: x.month)
  df['Dia'] = df['Fecha'].transform(lambda x: x.day)

#agregarFechas(df_silkroad_items, 'timestamp')

# Armo los data sets

In [8]:
df_train = df_silkroad_items.copy()
limite = 10.00
valor_BTC = 400
df_train['price'] = df_train['price'].transform(lambda x:( x*valor_BTC if x < limite else limite*valor_BTC))

In [9]:
limite = int(len(df_train)*0.7)

train = df_train[:limite]
test = df_train[limite:]

#-----------------------------


X_train_text = train['title']
Y_train = train['price']
del train

Y_test = test['price']
X_test_text = test['title']
del test

# Doc vs Term
count_vect = CountVectorizer()
X_train_text = count_vect.fit_transform(X_train_text)
X_test_text = count_vect.transform(X_test_text)

# transform  to a normalized tf-idf representation 
tfidf_transformer = TfidfTransformer()
X_train_text = tfidf_transformer.fit_transform(X_train_text)
X_test_text = tfidf_transformer.transform(X_test_text)

# Modelo

In [10]:
def linearRegresion_text():
  
  model = linear_model.LinearRegression()
  model.fit(X_train_text, Y_train)

  preds = model.predict(X_test_text)
  
  score_train = model.score(X_train_text, Y_train)
  score_test = model.score(X_test_text, Y_test)
  rmse = np.sqrt(mean_squared_error(Y_test, preds))

  print("RMSE: %f" % (rmse))
  print("Score en el set de train: {:.2f}%".format(score_train * 100))
  print("Score en el set de test: {:.2f}%".format(score_test * 100))
  return model

linearRegresion = linearRegresion_text()

RMSE: 609.452304
Score en el set de train: 65.46%
Score en el set de test: 58.46%


In [13]:
def XGBTexto():

  xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                    max_depth = 7, alpha = 10, n_estimators = 100)

  data_dmatrix = xgb.DMatrix(data=X_train_text,label=Y_train)
  xgb_reg.fit(X_train_text, Y_train)
  predsXGB = xgb_reg.predict(X_test_text)
  
  score_train = xgb_reg.score(X_train_text, Y_train)
  score_test = xgb_reg.score(X_test_text, Y_test)
  rmse = np.sqrt(mean_squared_error(Y_test, predsXGB))

  print("RMSE: %f" % (rmse))
  print("Score en el set de train: {:.2f}%".format(score_train * 100))
  print("Score en el set de test: {:.2f}%".format(score_test * 100))

  return xgb_reg
  
xgb_reg = XGBTexto()

RMSE: 694.005829
Score en el set de train: 57.14%
Score en el set de test: 46.13%


# Test

In [14]:
#archivo versus https://drive.google.com/file/d/1bKCo4VeH1owB8jGlwIT6NlljsbCSHs0X/view?usp=sharing

id_versus = '1bKCo4VeH1owB8jGlwIT6NlljsbCSHs0X'
downloaded = drive.CreateFile({'id': id_versus})
downloaded.GetContentFile('versus.csv')
df_test = pd.read_csv('versus.csv')
df_test.head()


,title,warnings,Type,Category,From,Stock,Sales,price,price_unit,timestamp
0,1oz. Distillate(FLAVORED),promoted,Physical,Concentrates,United States of America,88,9,115.96 EUR,per 1oz.,2021-10-24T04:07:09.509623
1,***3G for the price of 2G***,promoted,Physical,Cocaine,United Kingdom of Great Britain,Unlimited,5,70.91 EUR,per per unit,2021-10-24T04:07:09.509623
2,RAW THC DISTILLATE 1L (1000ml) $3 per gram Don...,promoted,Physical,Concentrates,United States of America,50,0,2576.90 EUR,per 1000ml / 1 Liter,2021-10-24T04:07:09.509623
3,**Free** Colombian Coke Giveaway! 3.5G + 0.5G ...,promoted,Physical,Cocaine,United Kingdom of Great Britain,32,10,277.76 EUR,per 3.5G + 0.5G Free,2021-10-24T04:07:09.509623
4,***OPENING SALE 3.5G PURE CHAMPAGNE MDMA***,promoted,Physical,MDMA,United States of America,447,2,128.84 EUR,per gram,2021-10-24T04:07:09.509623


In [15]:
#agregarFechas(df_test, 'timestamp')
df_test['price'] = df_test['price'].str.replace('EUR','')
df_test['price'] = df_test['price'].transform(lambda x: float(x))
#df_test

In [16]:
def test(model, count_vect, tfidf_transformer, df_test):
  
  df_train = df_test['title']
  df_test = df_test['price']
  df_train = count_vect.transform(df_train)
  df_train = tfidf_transformer.transform(df_train)

  score_train = model.score(df_train, df_test)
  score_test = model.score(df_train, df_test)
  preds = model.predict(df_train)
  rmse = np.sqrt(mean_squared_error(df_test, preds))

  print("RMSE: %f" % (rmse))
  print("Score en el set de train: {:.2f}%".format(score_train * 100))
  print("Score en el set de test: {:.2f}%".format(score_test * 100))

test(xgb_reg, count_vect, tfidf_transformer,df_test)

RMSE: 8462.606489
Score en el set de train: 0.51%
Score en el set de test: 0.51%


In [17]:
test(linearRegresion, count_vect, tfidf_transformer,df_test)

RMSE: 8458.794244
Score en el set de train: 0.60%
Score en el set de test: 0.60%
